In [4]:
import os
import TaxSolver as tx
from TaxSolver.constraints.income_constraint import IncomeConstraint
from TaxSolver.constraints.budget_constraint import BudgetConstraint
from TaxSolver.data_wrangling.data_loader import DataLoader
from TaxSolver.data_wrangling.bracket_input import BracketInput
from TaxSolver.objective import BudgetObjective

# Go to the project root directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

# Load taxpayer data from a system with 6 brackets into TaxSolver format
dl = DataLoader(path=os.path.join("data", "example", "simple_simul_1000.xlsx"), 
                income_before_tax="income_before_tax", 
                income_after_tax="income_after_tax")

# Initialize the solver
tax_solver = tx.TaxSolver(dl.households)

# Split input along inflection points to construct brackets
BracketInput.add_split_variables_to_solver(
    tx=tax_solver,
    target_var="income_before_tax",
    inflection_points=[0, 25_000, 50_000, 75_000, 100_000, 125_000, 150_000],
    group_vars=["k_everybody"]
)

# Define solver variables for the optimization
income_tax = tx.BracketRule(
    name="income_before_tax",
    var_name="income_before_tax",
    k_group_var="k_everybody",
    ub=1,
    lb=0,
)

# Add solver variables to the solver
tax_solver.add_rules([income_tax])

# Define constraints for the optimization
income_constraint = IncomeConstraint(0.0001, dl.households.values()) # No one can experience income shocks of more than 0.01%
budget_constraint = BudgetConstraint(
    "All_households", dl.households.values(), 0, 0 # The total tax revenue cannot decrease or increase
)

# Add constraints to the solver
tax_solver.add_constraints([income_constraint, budget_constraint])

# Solve the optimization problem
tax_solver.solve()

# Add an objective to the optimization
tax_solver.add_objective(BudgetObjective(budget_constraint))

# View results
print(tax_solver.rules_and_rates_table())

Loading data from Excel file...


AssertionError: income_after_tax column not in the dataframe. Aborting.